In [1]:
from keras.models import load_model, Model
from keras.preprocessing import image
from keras import models
import numpy as np
import pandas as pd
import os

acc = '91.0'

model = load_model('F:\\얼굴표정\\2.model_save\\face_model_' + acc + '.h5')

flatten = model.get_layer('flatten')
test_model = Model(inputs=model.input, outputs=flatten.output)

Using TensorFlow backend.
W0506 17:42:45.112956 18108 deprecation_wrapper.py:119] From C:\Users\Gachon\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0506 17:42:45.128908 18108 deprecation_wrapper.py:119] From C:\Users\Gachon\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0506 17:42:45.144822 18108 deprecation_wrapper.py:119] From C:\Users\Gachon\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0506 17:42:45.334373 18108 deprecation_wrapper.py:119] From C:\Users\Gachon\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0506 17:42:45.335370 18108 deprecation_wrap

In [3]:
abnormal_img_dir = 'F:\\얼굴표정\\1.datasets\\total\\sad'
normal_img_dir = 'F:\\얼굴표정\\1.datasets\\total\\happy'

abnormal_img = []
for img_name in os.listdir(abnormal_img_dir):
    img_path = os.path.join(abnormal_img_dir, img_name)
    img = image.load_img(img_path, target_size=(150, 150))
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.
    abnormal_img.append(img_tensor)
    
normal_img = []
for img_name in os.listdir(normal_img_dir):
    img_path = os.path.join(normal_img_dir, img_name)
    img = image.load_img(img_path, target_size=(150, 150))
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.
    normal_img.append(img_tensor)
    
abnormal_labels = np.ones(len(abnormal_img))
normal_labels = np.zeros(len(normal_img))
total_labels = np.concatenate((abnormal_labels, normal_labels), axis=0)

In [4]:
img_features = []
for img in abnormal_img:
    img_features.append(test_model.predict(img)[0])
    
for img in normal_img:
    img_features.append(test_model.predict(img)[0])
    
print(len(img_features))

3890


In [ ]:
# length of image features
print(len(img_features[0]))

# Dataset + Label
feature_set = []
for i in range(len(total_labels)):
    temp = np.append(img_features[i], total_labels[i])
    feature_set.append(temp)

# Feature column name
features_column = []
for i in range(len(img_features[0])):
    features_column.append('img_f' + str(i))

features_column.append('AA')

df = pd.DataFrame(feature_set, columns = features_column)
df.to_csv("E:\\Heartsound_VGG16\\Heartsound_VGG16_features\\img_features_" + acc + ".csv", index=False)